In [2]:
import tempfile
import json
import sys
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
import torch
import transformers
import numpy as np
# from transformers import AutoModelForCausalLM, AutoTokenizer
# cns_path_llama=r"/cns/iq-d/home/kkaizh/Gemma/gemma-1.1-2b-it"

from google3.pyglib import gfile

device = torch.device("cuda")
CELL = 'iq'
USER = 'kkaizh'
REPO= 'Gemma/gemma-1.1-2b-it' # option:Gemma/gemma-1.1-2b-it, llama/llama-7b
cache_dir = tempfile.gettempdir()
model_repo = f'/cns/{CELL}-d/home/{USER}/{REPO}'

# test loading from checkpoint
ckp_path=r"/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/instruction_tuning/checkpoint-500"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_repo, cache_dir=cache_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
model = transformers.AutoModelForCausalLM.from_pretrained(model_repo, cache_dir=cache_dir).to(device)

generation_config = transformers.GenerationConfig(
        max_length=2048,
        renomalize_logits=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_scores=True,
        output_logits=True,
    )

/export/hda3/borglet/remote_hdd_fs_dirs/0.colab_kernel_pytorch_gpu_kkaizh.kernel.kkaizh.3453488504364.14b334fb3717c109/mount/server/kernel.runfiles/google3/third_party/py/transformers/utils/generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/export/hda3/borglet/remote_hdd_fs_dirs/0.colab_kernel_pytorch_gpu_kkaizh.kernel.kkaizh.3453488504364.14b334fb3717c109/mount/server/kernel.runfiles/google3/third_party/py/transformers/utils/generic.py:339: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/export/hda3/borglet/remote_hdd_fs_dirs/0.colab_kernel_pytorch_gpu_kkaizh.kernel.kkaizh.3453488504364.14b334fb3717c109/mount/server/kernel.runfiles/google3/third_party/py/transformers/utils/generic.py:339: UserWarning: torch.utils._pytree._register

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
num_parameters = model.num_parameters()
total_size = num_parameters * 4  # Assuming 4 bytes per float32 parameter
print("Model size:", total_size / 1024**2, "MB")

Model size: 9560.2890625 MB


# Ligots-based generation test

In [ ]:
# generation test
import numpy as np
torch.cuda.empty_cache()
input_test = tokenizer("This is a test, please say hello to me", return_tensors="pt", max_length=128)
output = model.generate(input_test["input_ids"].to(device), generation_config=generation_config)
transition_scores = model.compute_transition_scores(
    output.sequences, output.scores, normalize_logits=True
)
# input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
# encoder-decoder models, like BART or T5.
input_length = 1 if model.config.is_encoder_decoder else input_test.input_ids.shape[1]
generated_tokens = output.sequences[:, input_length:]
print(tokenizer.decode(generated_tokens[0]))
print(np.mean(np.exp(transition_scores.cpu().numpy())))
for score in transition_scores[0]:
    # | token | token string | log probability | probability
    print(f" {score.cpu().numpy():.3f} | {np.exp(score.cpu().numpy()):.2%}")

! 👋

Hello! 👋 It's great to hear from you. How can I assist you today? 😊<eos>
0.8459854
 -0.491 | 61.19%
 -0.642 | 52.62%
 -0.152 | 85.88%
 -0.020 | 98.07%
 -0.505 | 60.38%
 -0.017 | 98.31%
 -0.378 | 68.53%
 -0.001 | 99.92%
 -0.000 | 100.00%
 -0.730 | 48.17%
 -0.001 | 99.94%
 -0.258 | 77.30%
 -0.000 | 100.00%
 -0.000 | 99.98%
 -0.009 | 99.06%
 -0.216 | 80.54%
 -0.097 | 90.75%
 -0.000 | 100.00%
 -0.540 | 58.25%
 -0.000 | 99.99%
 -0.000 | 99.95%
 -0.000 | 100.00%
 -0.663 | 51.55%
 -0.000 | 99.99%


# GSM8K dataset experiment

In [ ]:
# problem solving test
prompt_test = r'''Assume you're a student working on some mathematical problems. Now, you'll be giving mathematical problems, you need to do two tasks: a. Check if the question contains personal information (e.g., names etc.), output Yes or No only;\n
b. Solve this question; \n

Herer are some examples:
Question: Hector purchased a container of gumballs. He gave 4 to Todd, then he gave twice as many as he had given Todd to Alisha, and then he gave 5 less than four times as many to Bobby as he had given to Alisha. If Hector had 6 gumballs remaining, what is the total number of gumballs that Hector purchased? \n
Output:
a. Contains Personal Information: Yes
b. Answer: Hector gave to Alisha twice as many as he had given Todd, for a total of 4*2=<<4*2=8>>8 gumballs, Hector gave 5 less than four times as many to Bobby as he had given to Alisha, or a total of (8*4)-5=<<8*4-5=27>>27 gumballs. If Hector had 6 gumballs remaining, he originally purchased 4+8+27+6=<<4+8+27+6=45>>45 gumballs. #### 45

Case:
Auestion: Hector purchased a container of gumballs. He gave 4 to Todd, then he gave twice as many as he had given Todd to Alisha, and then he gave 5 less than four times as many to Bobby as he had given to Alisha. If Hector had 6 gumballs remaining, what is the total number of gumballs that Hector purchased?\n
Output:'''
test_ = r'''
a. Contains Personal Information: Yes
b. Answer: Hector gave to Alisha twice as many as he had given Todd, for a total of 4*2=<<4*2=8>>8 gumballs, Hector gave 5 less than four times as many to Bobby as he had given to Alisha, or a total of (8*4)-5=<<8*4-5=27>>27 gumballs. If Hector had 6 gumballs remaining, he originally purchased 4+8+27+6=<<4+8+27+6=45>>45 gumballs. #### 45
'''
tokenized_prompt_test = tokenizer(test_, return_tensors="pt", max_length=1024)
prompt_len = 1 if model.config.is_encoder_decoder else tokenized_prompt_test.input_ids.shape[1]
output_answer = model.generate(tokenized_prompt_test["input_ids"].to(device),generation_config=generation_config)
generated_answer_tokens = output_answer.sequences[:, prompt_len:]
decoded_str = tokenizer.decode(generated_answer_tokens[0])
transition_scores_ = model.compute_transition_scores(
    output_answer.sequences, output_answer.scores, normalize_logits=True
)
print(np.mean(np.exp(transition_scores_.cpu().numpy())))
print(prompt_len)
print(decoded_str)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0.8837341
143
The answer is 45<eos>


In [ ]:
# self-critque test
# self_check_prompt = r'''Assume you're a scorer. Given a correct solution: {correct_answer}\n
# and a solution by a student: {generated_ans}\n
# Please check if the student's solution is correct or not'''
# generated_answer = decoded_str
# golden_answer = "It rained 3+4=<<3+4=7>>7 inches So he collected 7*15=<<7*15=105>>105 gallons So he makes 105*1.2=$<<105*1.2=126>>126 from selling the water #### 126"
# self_check_test = self_check_prompt.format(correct_answer=golden_answer, generated_ans=generated_answer)
# tokenized_self_check_test = tokenizer(self_check_test, return_tensors="pt", truncation=True, max_length=512)
# self_check_len = tokenized_self_check_test['input_ids'].shape[1]
# output_answer = model.generate(tokenized_self_check_test["input_ids"].to(device), max_length=1024)
# print(self_check_len)
# print(tokenizer.decode(output_answer[0][self_check_len:]))

In [ ]:
# prompt tuning
prompt = """Assume you're a student working on some mathematical problems. Now, you'll be giving mathematical problems, you need to do four tasks: a. Check if the question contains personal information (e.g., names etc.), output "Contains personal information: Yes or No only";\n
b. Solve this question; \n
c. Do a self-critique on your answer quality, output your answer and your confidence level (low, moderate, high);\n
d. Identify if the question needs to be rewritten. If contains personal information and the confidence level is not high then output "Rewritten question: Yes" else output "Rewritten question: No";\n

Herer are some examples that teach you when to rewrite (answer confidence level is low/moderate and question contains personal information):
Question: Hector purchased a container of gumballs. He gave 4 to Todd, then he gave twice as many as he had given Todd to Alisha, and then he gave 5 less than four times as many to Bobby as he had given to Alisha. If Hector had 6 gumballs remaining, what is the total number of gumballs that Hector purchased? \n
Output:
a. Contains Personal Information: Yes
b. Answer: Hector gave to Alisha twice as many as he had given Todd, for a total of 4*2=<<4*2=8>>8 gumballs, Hector gave 5 less than four times as many to Bobby as he had given to Alisha, or a total of (8*4)-5=<<8*4-5=27>>27 gumballs. If Hector had 6 gumballs remaining, he originally purchased 4+8+27+6=<<4+8+27+6=45>>45 gumballs. #### 45.
c. Confidence Level: High
d. Rewritten question: No.

Question: A garden produced 237 potatoes, 60 fewer cucumbers and twice as many peppers than the cucumbers. How many vegetables did the garden produce? \n
Output:
a. Contains Personal Information: No
b. Answer: 237 potatoes + 60 cucumbers + 2*60 peppers = 237 + 60 + 120 = 317 vegetables.<eos>.
c. Confidence Level: Moderate
d. Rewritten question: No

Question: A boxer weighs 97 kg at 4 months from a fight. He is on a diet that allows him to lose 3 kg per month until the day of the fight. How much will he weigh on the day of the fight?
Output:
a. Contains Personal Information: No
b. Answer: 97 kg - 3 kg/month * 4 months = 97 kg.<eos>
c. Confidence Level: Low
d. Rewritten question: No

Question: Krystian works in the library. He borrows an average of 40 books every day. Every Friday, his number of borrowed books is about 40% higher than the daily average. How many books does he borrow in a week if the library is open from Monday to Friday? \n
Output:
a. Contains Personal Information: Yes
b. Answer: 40% higher than the daily average means 40% of 40 = 16 books. So Krystian borrows 40 + 16 = 56 books in a week.<eos>
c. Confidence Level: Low
d. Rewritten question: Yes

Question: Sally and Bob have made plans to go on a trip at the end of the year. They both decide to work as babysitters and save half of what they've earned for their trip. If Sally makes $6 per day and Bob makes $4 per day, how much money will they both have saved for their trip after a year? \n
Output:
a. Contains Personal Information: Yes
b. Answer: Sally and Bob will have saved $6*365=<<$6*365=22,150>>22,150 for their trip after a year.<eos>
c. Confidence Level: Moderate
d. Rewritten question: Yes

Your turn:
Auestion: {question}\n
Output:"""
question = "Irene earns $500 if she works for 40 hours a week and gets an extra $20 for every hour of overtime. If she worked 50 hours last week, calculate her total income."
input_ = tokenizer(prompt.format(question=question), return_tensors="pt", truncation=True, max_length=1536)
prompt_length = input_['input_ids'].shape[1]
print(prompt_length)
output_ = model.generate(input_["input_ids"].to(device), max_length=2048)
print(tokenizer.decode(output_[0][prompt_length:]))
# Two tasks - One is private or not / One is hybrid.

956

a. Contains Personal Information: Yes
b. Answer: Irene earned $500 for 40 hours, and an extra $20 for each hour of overtime. Therefore, Irene earned $500 + 20*5 = $500 + $100 = $600 for 50 hours of work.
c. Confidence Level: Moderate
d. Rewritten question: No<eos>


In [ ]:
raw_train_data = []
with gfile.Open(os.path.join("/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/datasets/GSM8K/train.jsonl"), "r", encoding="utf-8") as f:
  # raw_train_data = [json.loads(line) for line in f]
  for line in f:
    tmp = json.loads(line)
    tmp_data = {"input": tmp["question"], "label": tmp["answer"]}
    raw_train_data.append(tmp_data)

In [ ]:
raw_train_data[0]

{'input': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'output': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [ ]:
raw_train_data[0].keys()

dict_keys(['question', 'answer'])

# WMT22 Experiment

In [3]:
meta_path=r"/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/datasets/wmt22/"
file_name = ["europarl-v10.cs-en.pair.tsv", "europarl-v10.fr-en.pair.tsv", "europarl-v10.de-en.pair.tsv"]

In [4]:
# preprocess raw data
def process_raw(path):
  with gfile.Open(path, "r", encoding="utf-8") as f:
    data = []
    for line in f.readlines():
      _=line.split("\t")[:2]
      tmp={"src": _[0], "target": _[1]}
      data.append(tmp)
  return data

cs_en=process_raw(os.path.join(meta_path, file_name[0]))
fr_en=process_raw(os.path.join(meta_path, file_name[1]))
de_en=process_raw(os.path.join(meta_path, file_name[2]))

print(len(cs_en))
print(len(fr_en))
print(len(de_en))

645330
1915930
1828521


In [ ]:
print(cs_en[50])

{'src': 'To znamená, že co se mne týče, mám ohledně samotného principu mechanismu, jehož návrh máme před sebou, velké nejasnosti a pochyby.', 'target': 'This means that I, for my part, remain very puzzled and very doubtful as to the very principle of the mechanism that is proposed to us.'}


In [5]:
with gfile.Open(os.path.join(meta_path, "cs_en.json"), "w", encoding="utf-8") as f:
  json.dump(cs_en, f)
with gfile.Open(os.path.join(meta_path, "fr_en.json"), "w", encoding="utf-8") as f:
  json.dump(fr_en, f)
with gfile.Open(os.path.join(meta_path, "de_en.json"), "w", encoding="utf-8") as f:
  json.dump(de_en, f)

In [6]:
with gfile.Open(os.path.join(meta_path, "cs_en.json"), "r", encoding="utf-8") as f:
  test = json.load(f)

print(test[0])

{'src': '4.', 'target': '4.'}


In [ ]:
prompt_translate=r'''
Assume you're a professional translator, now please translate the following sentence into English. Output the translated sentence only.\n
Source sentence: {src}\n
Target sentence:
'''

In [ ]:
input_translate=tokenizer(prompt_translate.format(src=cs_en[66]["src"]), return_tensors="pt", truncation=True, max_length=256)
translate_length=input_translate['input_ids'].shape[1]
output_translate=model.generate(input_translate["input_ids"].to(device), max_length=512)
print(translate_length)
print(tokenizer.decode(output_translate[0][translate_length:]))
print(cs_en[66]["target"])

148
>The secondledem, which was decisive for my message, is that, as evidenced by the implementation of the Hague Program for the year 2007, the level of implementation, particularly in cooperation on criminal justice matters, was lower than in other areas, such as civil cooperation, border management, and immigration and asylum policies, which yielded positive results.<eos>
The second consideration on which I based my report is that, as may be seen from the report on the implementation of the Hague Programme for 2007, the level of implementation with regard to legal cooperation in the criminal field was somewhat low, even though satisfactory developments were registered in other sectors, such as civil co-operation, border management, immigration and asylum policies.


# MeQSum Experiment

In [ ]:
import pandas as pd

meqsum_path=r"/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/datasets/MedQSum/MeQSum.csv"
meqsum = pd.read_csv(meqsum_path)

In [ ]:
meqsum.head()

,File,CHQ,Summary
0,1-131188152.xml.txt,SUBJECT: who and where to get cetirizine - D\n...,Who manufactures cetirizine?
1,14348.txt,who makes bromocriptine\ni am wondering what c...,Who manufactures bromocriptine?
2,1-131985747.xml.txt,SUBJECT: nulytely\nMESSAGE: Hello can you tell...,"Who makes nulytely, and where can I buy it?"
3,15410.txt,Williams' syndrome\nI would like to have my da...,Where can I get genetic testing for william's ...
4,35.txt,ClinicalTrials.gov - Question - general inform...,Where can I get genetic testing for multiple m...


In [ ]:
print("The CHQ of 0-th data is:{0}".format(meqsum["CHQ"][0]))
print("The Summary of 0-th data is:{0}".format(meqsum["Summary"][0]))

The CHQ of 0-th data is:SUBJECT: who and where to get cetirizine - D
MESSAGE: I need/want to know who manufscturs Cetirizine. My Walmart is looking for a new supply and are not getting the recent
The Summary of 0-th data is:Who manufactures cetirizine?


In [ ]:
# meqsum_prompt = r'''
# Please summarize the below consumer health question (CHQ) following the given examples, output the summarized question only:\n

# Examples:
# CHQ: SUBJECT: who and where to get cetirizine - D
# MESSAGE: I need/want to know who manufscturs Cetirizine. My Walmart is looking for a new supply and are not getting the recent
# Summary: Who manufactures cetirizine?

# CHQ: who makes bromocriptine
# i am wondering what company makes the drug bromocriptine, i need it for a mass i have on my pituitary gland and the cost just keeps raising. i cannot ever buy a full prescription because of the price and i was told if i get a hold of the maker of the drug sometimes they offer coupons or something to help me afford the medicine. if i buy 10 pills in which i have to take 2 times a day it costs me 78.00. and that is how i have to buy them.  thanks.
# Summary: Who manufactures bromocriptine?

# Case:
# CHQ: {chq}\n
# Summary:
# '''
meqsum_prompt = r'''
Please summarize the below consumer health question (CHQ) into a simple question, output the summarized question only.

CHQ: {chq}\n
Summary:
'''

In [ ]:
tokenized_meqsum = tokenizer(meqsum_prompt.format(chq=meqsum["CHQ"][5]), return_tensors="pt", max_length=256)
prompt_len = 1 if model.config.is_encoder_decoder else tokenized_meqsum.input_ids.shape[1]
output_answer = model.generate(tokenized_meqsum["input_ids"].to(device),generation_config=generation_config)
generated_answer_tokens = output_answer.sequences[:, prompt_len:]
decoded_str = tokenizer.decode(generated_answer_tokens[0])
transition_scores_ = model.compute_transition_scores(
    output_answer.sequences, output_answer.scores, normalize_logits=True
)
print(np.mean(np.exp(transition_scores_.cpu().numpy())))
print(prompt_len)
print(decoded_str)

0.8329723
91
The consumer is seeking information about commercial genetic tests for the IHHS heart condition in Texas.<eos>


In [ ]:
def obtain_response_logits(CHQ, meqsum_prompt):
  tokenized_meqsum = tokenizer(meqsum_prompt.format(chq=CHQ), return_tensors="pt", max_length=512)
  prompt_len = 1 if model.config.is_encoder_decoder else tokenized_meqsum.input_ids.shape[1]
  output_answer = model.generate(tokenized_meqsum["input_ids"].to(device),generation_config=generation_config)
  generated_answer_tokens = output_answer.sequences[:, prompt_len:]
  decoded_str = tokenizer.decode(generated_answer_tokens[0])
  transition_scores = model.compute_transition_scores(
      output_answer.sequences, output_answer.scores, normalize_logits=True
  )
  raw_logits = np.exp(transition_scores.cpu().numpy())
  mean_logits = np.mean(raw_logits)
  median_logits = np.median(raw_logits)
  quantile25 = np.quantile(raw_logits, 0.25)
  quantile50 = np.quantile(raw_logits, 0.50)
  quantile75 = np.quantile(raw_logits, 0.75)
  logits = {"mean": str(mean_logits), "median": str(median_logits), "quantile25": str(quantile25), "quantile50": str(quantile50), "quantile75": str(quantile75)}

  return {"chq": CHQ, "generated_summary": decoded_str, "logits": logits}

In [ ]:
obtain_response_logits(meqsum["CHQ"][5], meqsum_prompt)

{'chq': 'Genetic Test for IHHS heart condition. Is there a commercial genetic test for the IHHS heart condition?  My family suffers from this heridity condition and I would like to know who is susseptable.   If so, where could I get it done in [LOCATION] Texas?',
 'generated_summary': 'The consumer is seeking information about commercial genetic tests for the IHHS heart condition in Texas.<eos>',
 'logits': {'mean': '0.8329723',
  'median': '0.9106504',
  'quantile25': '0.7449459433555603',
  'quantile50': '0.910650372505188',
  'quantile75': '0.9959591925144196'}}

In [ ]:
meqsum_results = []
for i in range(len(meqsum)):
  meqsum_results.append(obtain_response_logits(meqsum["CHQ"][i], meqsum_prompt))

with gfile.Open(r"/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/datasets/MedQSum/meqsum_0shot.json", "w", encoding="utf-8") as f:
  json.dump(meqsum_results, f)

## Obtain safety check

In [ ]:
def obtain_safety_response(CHQ, safety_check_prompt):
  tokenized_meqsum = tokenizer(safety_check_prompt.format(chq=CHQ), return_tensors="pt", max_length=512)
  prompt_len = 1 if model.config.is_encoder_decoder else tokenized_meqsum.input_ids.shape[1]
  output_answer = model.generate(tokenized_meqsum["input_ids"].to(device),generation_config=generation_config)
  generated_answer_tokens = output_answer.sequences[:, prompt_len:]
  decoded_str = tokenizer.decode(generated_answer_tokens[0])
  if "Yes" in decoded_str:
    return 1
  else:
    return 0

In [ ]:
# safety_check_prompt=r'''
# DO you think there is privacy concern in the following consumer health question (anything involves personal information considered as privacy)? Answer Yes or No only. \n
# CHQ: {chq}\n
# '''

safety_check_prompt=r'''
DO you think there is privacy concern in the following consumer health question (anything involves personal information considered as privacy)? Answer Yes or No only. \n

Here are some examples:
CHQ: SUBJECT: who and where to get cetirizine - D
MESSAGE: I need/want to know who manufscturs Cetirizine. My Walmart is looking for a new supply and are not getting the recent
Answer: No

CHQ: SUBJECT: who makes bromocriptine
MESSAGE: I am wondering what company makes the drug bromocriptine, i need it for a mass i have on my pituitary gland and the cost just keeps raising. i cannot ever buy a full prescription because of the price and i was told if i get a hold of the maker of the drug sometimes they offer coupons or something to help me afford the medicine. if i buy 10 pills in which i have to take 2 times a day it costs me 78.00. and that is how i have to buy them.  thanks.
Answer: Yes

CHQ: {chq}\n
'''

In [ ]:
# load results
with gfile.Open(r"/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/datasets/MedQSum/meqsum_0shot.json", "r") as f:
  meqsum_0shot = json.load(f)

with gfile.Open(r"/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/datasets/MedQSum/meqsum_2shot.json", "r") as f:
  meqsum_2shot = json.load(f)

In [ ]:
meqsum_0shot[0]

{'chq': 'SUBJECT: who and where to get cetirizine - D\nMESSAGE: I need/want to know who manufscturs Cetirizine. My Walmart is looking for a new supply and are not getting the recent',
 'generated_summary': '- Cetirizine is a common over-the-counter (OTC) medication used to treat allergies and cold symptoms.\n- It is manufactured by multiple companies.\n- Walmart is looking for a new supplier for Cetirizine.\n\nPlease provide a concise and understandable summary of the CHQ.<eos>',
 'logits': {'mean': '0.8311022',
  'median': '0.9056625',
  'quantile25': '0.7018575668334961',
  'quantile50': '0.905662477016449',
  'quantile75': '0.9983581900596619'}}

In [ ]:
for i in range(len(meqsum_0shot)):
  meqsum_0shot[i]["safety_check"]=obtain_safety_response(meqsum_0shot[i]["chq"], safety_check_prompt)

with gfile.Open(r"/cns/iq-d/home/kkaizh/multi_obj_cascade_llm/datasets/MedQSum/meqsum_2shot_satefy_check_.json", "w", encoding="utf-8") as f:
  json.dump(meqsum_0shot, f)

In [ ]:
safety_count = [point["safety_check"] for point in meqsum_0shot]
print(safety_count.count(1))

830


In [ ]:
print(safety_count)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 

# Training Test

In [ ]:
import numpy as np

def tokenize(text):
  """
  Simple tokenizer that splits text by whitespace.

  Args:
      text: A string to be tokenized.

  Returns:
      A list of tokens.
  """
  return text.lower().split()

def ngrams(sequence, n):
  """
  Generates n-grams from a sequence.

  Args:
      sequence: A list of tokens.
      n: The n-gram size.

  Returns:
      A list of n-grams (subsequences of length n).
  """
  return [sequence[i:i+n] for i in range(len(sequence) - n + 1)]

def rouge_score(hypotheses, references, n=1):
  """
  Calculates basic ROUGE-N score (precision) between hypotheses and references.

  Args:
      hypotheses: A list of strings representing the hypothesis sequences.
      references: A list of lists of strings representing the reference sequences
                  for each hypothesis.
      n: The n-gram size (default: 1 for ROUGE-1).

  Returns:
      A list of ROUGE-N scores for each hypothesis-reference pair.
  """

  scores = []
  for hyp, refs in zip(hypotheses, references):
    hyp_ngrams = set(ngrams(tokenize(hyp), n))
    ref_ngrams_count = np.zeros((len(refs), len(hyp_ngrams)))

    for i, ref in enumerate(refs):
      for ngram in ngrams(tokenize(ref), n):
        if ngram in hyp_ngrams:
          ref_ngrams_count[i, hyp_ngrams.index(ngram)] += 1

    max_ref_ngrams = np.max(ref_ngrams_count, axis=0)
    precision = np.mean([np.sum(ngrams) / max(len(ngrams), 1) for ngrams in ref_ngrams_count])
    scores.append(precision)

  return scores

In [ ]:
# Setup training arguments.
def get_training_args(
    num_epochs: int,
    batch_size: int,
    learning_rate: float,
    weight_decay: float,
) -> transformers.Seq2SeqTrainingArguments:
  return transformers.Seq2SeqTrainingArguments(
      warmup_steps=10,
      logging_steps=100,
      fp16=True,
      learning_rate=learning_rate,
      weight_decay=weight_decay,
      num_train_epochs=num_epochs,
      logging_dir='logs/',
      output_dir='results/',
      per_device_eval_batch_size=batch_size,
      per_device_train_batch_size=batch_size,
      save_strategy='epoch',
      evaluation_strategy='epoch',
      label_names=['labels'],
      save_total_limit=2,
      load_best_model_at_end=True,
      predict_with_generate=True,
      report_to=[],
  )

# Setup trainer.
def get_trainer(model: transformers.AutoModelForCausalLM,
    tokenizer: transformers.AutoTokenizer,
    training_args: transformers.Seq2SeqTrainingArguments,
    train_dataset,
    eval_dataset,
) -> transformers.Seq2SeqTrainer:
  # customized compute metrics
  def compute_metrics(eval_pred):
    # print("Enter Evaluation")
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE-N score
    result = rouge_score(hypotheses=decoded_preds, references=decoded_labels)

    # print(result)
    return {"rouge": result}

  # def data_collator(data):
  #   passed_data = {'input_ids': torch.stack([f[0] for f in data]), 'attention_mask': torch.stack([f[1] for f in data]), 'labels': torch.stack([f[2] for f in data])}
  #   print(passed_data)
  #   x, y, z = passed_data['input_ids'].size()
  #   print(x, y, z)
  #   normalizer = torch.tensor(2048**0.5)
  #   tmp = passed_data["input_ids"] * normalizer
  #   print(tmp.size())
  #   return passed_data

  data_collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='pt', padding="max_length", max_length=64)

  return transformers.Seq2SeqTrainer(
      model=model,
      tokenizer=tokenizer,
      args=training_args,
      train_dataset=train_dataset,
      # eval_dataset=eval_dataset,
      # compute_metrics = compute_metrics,
      # callbacks=[transformers.EarlyStoppingCallback(early_stopping_patience=3)],
      # data_collator = lambda data: {'input_ids': torch.stack([f[0] for f in data]), 'attention_mask': torch.stack([f[1] for f in data]), 'labels': torch.stack([f[2] for f in data])}
      data_collator=data_collator
    )

In [ ]:
from torch.utils.data import Dataset

class my_dataset(Dataset):
  def __init__(self, raw_data, tokenizer, max_length):
    self.raw_data = raw_data
    self.tokenizer = tokenizer
    self.max_length = max_length
    self.max_target_length = max_length

  def __len__(self):
    return len(self.raw_data)

  def __getitem__(self, idx):
    data = self.raw_data[idx]
    input_text = data['question']
    target_text = data['answer']
    input = self.tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length)
    # input_ids = torch.tensor(input["input_ids"])
    # attn_masks = torch.tensor(input["attention_mask"])
    input_ids = input["input_ids"]
    attn_masks = input["attention_mask"]
    target = self.tokenizer(target_text, padding="max_length", truncation=True, max_length=self.max_target_length)
    # labels = torch.tensor(target["input_ids"])
    labels = target["input_ids"]
    return {"input_ids": input_ids, "attention_mask": attn_masks, "labels": labels}

train_dataset = my_dataset(raw_train_data[:20], tokenizer, 64)
eval_dataset = my_dataset(raw_train_data[50:60], tokenizer, 64)

training_args = get_training_args(num_epochs=2, batch_size=1, learning_rate=0.001, weight_decay=0.2)
trainer = get_trainer(model, tokenizer, training_args, train_dataset, eval_dataset)
# torch.cuda.empty_cache()
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 